# Total Nappy Changes over Time

Build a series of cumulative nappy changes over time. Resampled to Daily.

In [1]:
# import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# load data and check

df = pd.read_csv("../Data/data.csv")
df["DateTime"] = pd.to_datetime(df["DateTime"])

df.set_index("DateTime", inplace=True)

df.head(10)

,Feed,Poo,Wee,Vomit
DateTime,,,,
2022-07-30 13:00:00,True,True,True,False
2022-07-30 17:00:00,True,False,True,False
2022-07-30 19:45:00,True,False,True,False
2022-07-30 22:20:00,True,True,False,False
2022-07-31 01:25:00,True,True,True,False
2022-07-31 05:30:00,True,False,True,False
2022-07-31 09:45:00,True,False,True,False
2022-07-31 14:00:00,True,False,True,False
2022-07-31 18:30:00,True,False,True,False


In [3]:
# Create series of nappy change. Nappy change is when poo or wee has occurred at a timestamp.

# poo or wee data
array_poo_wee = df[["Poo", "Wee"]].values

# use list comprehension: if either poo or wee is True, return True otherwise return False then convert to pd Series
nappy_changes = [True if x[0] == True or x[1] == True else False for x in array_poo_wee]
nappy_changes = pd.Series(data=nappy_changes)

# Length of nappy_changes must equal number of datetime stamps
print(f"Length of nappy_changes == df index length: {len(nappy_changes) == df.shape[0]}")


Length of nappy_changes == df index length: True


In [4]:
# create data frame for nappy_changes
df_nappy_changes = pd.DataFrame(data={"DateTime": df.index,
                    "Nappy Change": nappy_changes})

df_nappy_changes.set_index("DateTime", inplace=True)

# resample down to daily
df_nappy_changes_daily = df_nappy_changes.resample("D").sum()

# add cumulative sum column
df_nappy_changes_daily["Count"] = df_nappy_changes_daily["Nappy Change"].cumsum()

# drop nappy change column and reset index to save as json
df_nappy_changes_daily.drop("Nappy Change", axis=1, inplace=True)
df_nappy_changes_daily = df_nappy_changes_daily.reset_index()
df_nappy_changes_daily.rename(columns={"DateTime": "Date"}, inplace=True)

# Check dataframe
df_nappy_changes_daily

,Date,Count
0,2022-07-30,4
1,2022-07-31,10
2,2022-08-01,15
3,2022-08-02,20
4,2022-08-03,26
...,...,...
378,2023-08-12,2269
379,2023-08-13,2271
380,2023-08-14,2273
381,2023-08-15,2275


In [5]:
# Save to json
df_nappy_changes_daily.to_json(path_or_buf="../Data/Website_Data/nappy_changes.json", orient="records", date_format="iso")

In [6]:
# get average daily nappies

days = df_nappy_changes_daily.shape[0]

total_changes = df_nappy_changes_daily.iat[days - 1, 1]

print(f"{total_changes/days:.3}")


5.96
